# Data Cleaning and Title Optimization

## Table of Contents
1. [Introduction](#introduction)
2. [The Dataset](#the-dataset)
3. [Data Cleaning Process](#data-cleaning)
4. [Title Optimization Process](#title-optimization)
5. [Conclusion](#conclusion)

### Libraries

In [63]:
import pandas as pd
import numpy as np
import re

## 1. Introduction <a id="introduction"></a>

### Objective:
The goal of this project is to clean raw product data and optimize product titles for better marketing impact. This involves:
- Identifying and resolving data quality issues.
- Standardizing formats and handling missing values.
- Creating a new short_title feature to improve SEO and readability.

### 2. The Dataset <a id="the-dataset"></a>

In [64]:
path = "C:/Users/Alfred Zimo/OneDrive/Desktop/desktop stuff/Data Science/HGN12/Stage 1/productdata.xlsx"
df = pd.read_excel(path)
df.head()

,PRODUCTID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCTTYPEID,ProductLength
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650.0,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755.0,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537.0,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996.0,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112.0,598.424000


The dataset consist of 3,847 rows and 6 columns of product related information, including:
- PRODUCTID: and identifier for each product
- TITLE: full product title
- BULLET_POINTS: product features
- DESCRIPTION: Product description
- PRODUCTTYPEID: category 
- ProductLength: measurement

## 3. Data Cleaning Process <a id="data-cleaning"></a>

### Handling missing values

In [65]:
missing_values = df.isnull().sum()
missing_values

PRODUCTID           0
TITLE               0
BULLET_POINTS    1591
DESCRIPTION      2144
PRODUCTTYPEID     178
ProductLength     178
dtype: int64

Filling missing values of categorical columns

In [66]:
df["BULLET_POINTS"].fillna("Not Available", inplace=True)
df["DESCRIPTION"].fillna("Not Available", inplace=True)

Handling missing values of numerical columns

In [67]:
df["PRODUCTTYPEID"].fillna(df["PRODUCTTYPEID"].median(), inplace=True)
df["ProductLength"].fillna(df["ProductLength"].median(), inplace=True)

In [68]:
df.isnull().sum()

PRODUCTID        0
TITLE            0
BULLET_POINTS    0
DESCRIPTION      0
PRODUCTTYPEID    0
ProductLength    0
dtype: int64

### Handling duplicates

In [69]:
duplicates = df.duplicated().sum()
duplicates

217

In [70]:
df = df.drop_duplicates()

In [71]:
df.duplicated().sum()

0

### Verifying Data Accuracy

Check invalid negative prices

In [72]:
invalid_product_length = (df["ProductLength"] < 0).sum()

### Standardize column names

The aim is to standardize the column names for consistency

In [73]:
df.rename(columns={
    "ProductLength": "product_length",
    "PRODUCTTYPEID": "product_type_id",
    "PRODUCTID": "product_id",}, inplace=True)

In [74]:
df.columns = [col.lower() for col in df.columns]

In [75]:
df.head(2)

,product_id,title,bullet_points,description,product_type_id,product_length
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,Not Available,1650.0,2125.98
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",Not Available,2755.0,393.70


All missing values have been handled, duplicates removed, and column names standardized

### 4. Title Optimization Process <a id="title-optimization"></a>

The objective is to generate a concise version of the product titles that retains essential details and immproves readability for marketing and SEO purposes. 
- Each title has three elements: product name, category and attributes.
- We need to remove redundant words like "includes", "set of", etc.
- create new columns short_title
- Limit short_titles to 30-50 characters

In [76]:
def create_short_title(title):
    # Remove brand names
    brand_words = ["ArtzFolio", "Marks & Spencer", "PRIKNIK", "ALISHAH", "HINS", "PUMA", "Hexwell", "Delavala"]
    for brand in brand_words:
        title = re.sub(rf"\b{brand}\b", "", title, flags=re.IGNORECASE)

    # Remove redundant words/phrase
    redundant_phrases = [
        "for", "with", "and", "&", "includes", "set of", "features", "pack of", 
        "combo", "tie back", "fabric", "material", "made of", "door", 
        "window", "room", "kit", "series", "edition", "eyelets", "canvas", "pcs", "width",
        "inch", "feet", "cm", "meter", "ml", "litre", "oz", "g", "kg", "count", "includes", "features"
    ]
    for phrase in redundant_phrases:
        title = re.sub(rf"\b{phrase}\b", "", title, flags=re.IGNORECASE)

    # Remove product codes 
    title = re.sub(r"\b[A-Z0-9_]+\b", "", title)  

    # Remove unwanted special characters and multiple spaces
    title = re.sub(r"[\|\(\)\[\]\{\}]", "", title)
    title = re.sub(r"\s+", " ", title).strip()

    # Extract key words: 
    words = title.split()
    important_words = [word for word in words if len(word) > 2 or re.search(r'\d+', word)]

    # Ensure correct structure: Name + Category + Key Attribute
    short_title = " ".join(important_words[:6])

    # Final length constraint (30-50 characters)
    if len(short_title) > 50:
        short_title = short_title[:50].rsplit(' ', 1)[0]

    return short_title


In [77]:
# create short_title column
df["short_title"] = df["title"].apply(create_short_title)

df[["title", "short_title"]].head(20)

,title,short_title
0,ArtzFolio Tulip Flowers Blackout Curtain for D...,Tulip Flowers Blackout Curtain .5feet 54inch
1,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,Girls' Pyjama Sets T86_2561C_Navy Mix_9-
2,PRIKNIK Horn Red Electric Air Horn Compressor ...,Horn Red Electric Air Horn Compressor
3,ALISHAH Women's Cotton Ankle Length Leggings C...,Women's Cotton Ankle Length Leggings Plus
4,The United Empire Loyalists: A Chronicle of th...,The United Empire Loyalists: Chronicle the
5,HINS Metal Bucket Shape Plant Pot for Indoor &...,Metal Bucket Shape Plant Pot Indoor
6,Ungifted: My Life and Journey,Ungifted: Life Journey
7,Delavala Self Adhesive Kitchen Backsplash Wall...,"Self Adhesive Kitchen Backsplash Wallpaper, Oil"
8,PUMA Cali Sport Clean Women's Sneakers White L...,Cali Sport Clean Women's Sneakers White
9,Hexwell Essential oil for Home Fragrance Oil A...,Essential oil Home Fragrance Oil Aroma


In [78]:
df.head()

,product_id,title,bullet_points,description,product_type_id,product_length,short_title
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,Not Available,1650.0,2125.980000,Tulip Flowers Blackout Curtain .5feet 54inch
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",Not Available,2755.0,393.700000,Girls' Pyjama Sets T86_2561C_Navy Mix_9-
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537.0,748.031495,Horn Red Electric Air Horn Compressor
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996.0,787.401574,Women's Cotton Ankle Length Leggings Plus
4,283658,The United Empire Loyalists: A Chronicle of th...,Not Available,Not Available,6112.0,598.424000,The United Empire Loyalists: Chronicle the


### Conclusion

This report details the data cleaning and title optimization process for the given marketing dataset. The goal was to address data quality issues, standardize formats, and create a concise short_title feature for improved SEO and readability

In [79]:
cleaned_file_path = "cleaned_productdata.xlsx"
df.to_excel(cleaned_file_path, index=False)